In [1]:
import sys
sys.path.insert(1, '../')

import experiment
import pickle
from utils import *
from Architectures import *
import random, os
suppress_warnings()
EPOCHS = 256


def countp(model, params):
    tmp = model(**params)
    tmp.build(input_shape=(1,15,7))
    return tmp.count_params()


n_params = []

os.environ['PYTHONHASHSEED']=str(0)
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.0f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])


filename = '../data/data100k_raw_combined_atlas_cut.pkl'
n_experiments = 8
SUFFIX = '_latent_dim_edge_AeqB_only_1_2'


#TESTING ######
# EPOCHS = 2
# filename = '../data/data50k_raw_combined_atlas_cut_small.pkl'
# n_experiments = 2
# SUFFIX = '_latent_dim_edge_TEST'
###########

ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-conda-linux-gnu-c++  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0


Welcome to JupyROOT 6.22/08


2022-06-01 12:38:15.635894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 12:38:15.635941: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
PI = experiment.Experimenter(filename)

Initializing Experimenter
	Loading Data from ../data/data100k_raw_combined_atlas_cut.pkl
	Data Loaded
	Creating Splits
	Splits Created
Done initalizing


In [3]:
PI.data_loader('pairwise', gen_multijet_to_inv_dataset, class_weight_invariant, tf.constant, aux_params=dict(dR_keep=False, multijet_n=1))

2022-06-01 12:40:31.634024: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-01 12:40:31.634564: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-01 12:40:31.634621: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (top1.hep.utexas.edu): /proc/driver/nvidia/version does not exist
2022-06-01 12:40:31.636088: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
%%capture output
from datetime import datetime

import bisect 

scan = [1,3]
for i in scan:
    curr_auc = []
    print("\t\tLATENT DIM", 2**i)
    j=0
    tf.random.set_seed(42+j)
    np.random.seed(42+j)
    random.seed(42+j)
    tail_string = PI.get_tail_string({'depth':5, 'ec_widths':(64,128,256,128,2**i), 'width':64})
    classifier_name = '%s_%s'%('pairwise', tail_string)

    PI.train_classifier('pairwise', {'depth':5, 'ec_widths':(64,128,256,128,2**i), 'width':64}, epochs=EPOCHS, seed=42+j)

In [ ]:
output.show()

In [ ]:
latent2 = PI.models['pairwise_5_(64, 128, 256, 128, 2)_64']
latent8 = PI.models['pairwise_5_(64, 128, 256, 128, 8)_64']

In [ ]:
import seaborn as sns
def gen_tsne(curr_event, latent_label, text=r'\textbf{Latent Representation} in Pairwise Architecture'):
    c_cut = 5
    cmap = sns.cubehelix_palette(start=26/10, light=.97, as_cmap=True)

    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.size'] = 16
    plt.rcParams['figure.autolayout'] = False
    plt.rcParams['text.usetex'] = True

    ttH_loc = np.array([curr_event[i] for i in range(len(latent_label)) if latent_label[i][1]==1])
    g = sns.jointplot(x=ttH_loc[:,0], y = ttH_loc[:,1], color=cmap(100), space=0, label='ttH jets',
                      cmap=cmap, kind='kde', height=10, fill=True, cut=c_cut,
                     marginal_kws={'linewidth': 0.0, 'alpha':1.0})

    COL2 = '#000000'
    linew = 0.6
    nttH_loc = np.array([curr_event[i] for i in range(len(latent_label)) if latent_label[i][0]==1])
    sns.kdeplot(x=nttH_loc[:,0], y = nttH_loc[:,1], shade=False, label=r'ttbar jets',color=COL2, linewidths=linew, cut=c_cut, 
                levels=10, ax=g.ax_joint)

    sns.kdeplot(nttH_loc[:,0], ax=g.ax_marg_x, color=COL2, lw=linew)
    sns.kdeplot(y=nttH_loc[:,1], ax=g.ax_marg_y, color=COL2, lw=linew)

    ax = g.ax_joint

    import matplotlib.patches as  mpatches
    import matplotlib.lines as  mlines

    handles = [mpatches.Patch(facecolor=cmap(100), label=r'$tt(H\rightarrow\tau\tau)$ Events'),
               mlines.Line2D([], [], color=COL2, label=r'$t\overline{t}$ Events', lw=linew)]
    legend = ax.legend(loc='upper left', handles=handles, frameon=False, title=text)
    legend._legend_box.align = 'left'
    plt.setp(legend.get_texts(), color=cmap(0.98))
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.size'] = 16
    plt.rcParams['figure.autolayout'] = True



    ax.set_facecolor(cmap(c_cut))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    g.ax_marg_y.get_yaxis().set_visible(False)
    g.ax_marg_x.get_xaxis().set_visible(False)
    return g

import os

#to get latex to work 
os.environ['PATH'] = "%s:/usr/local/cuda-11.2/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/anaconda/bin:/home/delon/texlive/bin/x86_64-linux:/home/delon/.local/bin:/home/delon/bin"%os.environ['PATH']



In [ ]:
X_test, y_test = PI.get_test_dataset('pairwise')

In [ ]:
n_cut = int(len(X_test)*0.5)
indices = np.random.permutation(len(X_test))
cut = np.s_[indices[:n_cut]]
latent_label = y_test.numpy()[cut]

In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
def compute_tsne_embedded(latent_reps, perplexity=50):
    distance_matrix = pairwise_distances(latent_reps, latent_reps, metric='cosine', n_jobs=-1)
    latent_reps_embedded_tsne = TSNE(metric="precomputed", n_components=2, learning_rate='auto', 
                                      verbose=2, perplexity=perplexity, 
                                     n_iter=2000, n_jobs=-1)
    latent_reps_embedded = latent_reps_embedded_tsne.fit_transform(distance_matrix)
    return latent_reps_embedded

def compute_tsne(model, perplexity=50):
    latent_getter = LatentGetter(model.layers[0:3], condensed=True)
    latent_reps = latent_getter.predict(X_test.numpy()[cut])
    return compute_tsne_embedded(latent_reps, perplexity=perplexity)

In [ ]:
latent2_embedded = compute_tsne(latent2)

In [ ]:
ax = gen_tsne(latent2_embedded, latent_label, text=r'\textbf{Latent Representation} in $\ell=2$ Pairwise Architecture')
ax.savefig('figures/l2_tsne.pdf')

In [ ]:
latent8_embedded = compute_tsne(latent8)
ax = gen_tsne(latent8_embedded, latent_label, text=r'\textbf{Latent Representation} in $\ell=2^3$ Pairwise Architecture')
ax.savefig('figures/l8_tsne.pdf')

In [ ]:
X_test_ATLAS, y_test_ATLAS = gen_dataset_high_level(PI.events_test, PI.events_oup_test, PI.events_tag_test)

In [ ]:
ATLAS_embedded = compute_tsne_embedded(np.array(X_test_ATLAS)[cut])

In [ ]:
ax = gen_tsne(ATLAS_embedded, latent_label, text=r'\textbf{Latent Representation} in ATLAS Features')
ax.savefig('figures/ATLAS_tsne.pdf')